#### Demostración de info para hallar ángulo de fase de la fuente 

In [ ]:
with open("angle_test.atp", "r+") as f:
    lines = f.readlines()
lines_copy = lines.copy()
for line_idx, line in enumerate(lines):
    if "C MainSource" == line.strip("\n"):
        print(line_idx, line)
        print(lines[line_idx+1][0:8])
        print(lines[line_idx+1][8:16])
        print(f"16:24: {lines[line_idx+1][16:24]}")
        print(f"24:32: {lines[line_idx+1][24:32]}")
        print(f"32:40: {lines[line_idx+1][32:40]}")
        print(f"40:48: {lines[line_idx+1][40:48]}")
        print(f"48:56: {lines[line_idx+1][48:56]}")
        print(f"56:64: {lines[line_idx+1][56:64]}")
        print(f"64:72: {lines[line_idx+1][64:72]}")
        print(f"64:72: {lines[line_idx+1][64:72]}")
        print(f"72:80: {lines[line_idx+1][72:80]}")

        print(f"Ángulo de fase utiliza 3 digitos y 5 decimales (9 posiciones):")
        print(f"En las columnas [31:40]: {lines[line_idx+1][31:40]}")
        source_phase = lines[line_idx+1][31:40]

# print("C MainSource\n" in lines_copy)

In [ ]:
import random

phase_int = random.randint(0, 360)
phase_dec = random.random()
source_phase = phase_int + phase_dec

print(source_phase)
random_phase = True



In [ ]:
with open("atp_scenarios\\angle_test.atp") as f:
    lines = f.readlines()
lines_copy = lines.copy()

for line_idx, line in enumerate(lines):
    if "C MainSource" in line:
        d_mainSource = line_idx + 1
phase_line_a = lines_copy[d_mainSource]
phase_line_b = lines_copy[d_mainSource+1]
phase_line_c = lines_copy[d_mainSource+2]
phi_a = str(round(source_phase, 5))
phi_b = str(round(source_phase-120,5))
phi_c = str(round(source_phase-240,5))

if random_phase:
    lines_copy[d_mainSource] = f"{phase_line_a[:30]}{phi_a:>10}{phase_line_a[40:]}"
    lines_copy[d_mainSource+1] = f"{phase_line_b[:30]}{phi_b:>10}{phase_line_b[40:]}"
    lines_copy[d_mainSource+2] = f"{phase_line_c[:30]}{phi_c:>10}{phase_line_c[40:]}"
print(phase_line_a)
print(lines_copy[d_mainSource])
with open("archivoprueba.atp", "w") as f:
    f.writelines(lines_copy)

In [ ]:
try:
    lines_copy[alvaro]
except NameError:
    print("Check mainSource/microGridSource comment")



In [ ]:
with open("IEEE34_Base.atp", "r+") as f:
    lines = f.readlines()
lines_copy = lines.copy()
for line_idx, line in enumerate(lines):
    if "C MainSource" == line.strip("\n"):
        print(line_idx, line)
        print(lines[line_idx+1][0:8])
        print(lines[line_idx+1][8:16])
        print(f"16:24: {lines[line_idx+1][16:24]}")
        print(f"24:32: {lines[line_idx+1][24:32]}")
        print(f"32:40: {lines[line_idx+1][32:40]}")
        print(f"40:48: {lines[line_idx+1][40:48]}")
        print(f"48:56: {lines[line_idx+1][48:56]}")
        print(f"56:64: {lines[line_idx+1][56:64]}")
        print(f"64:72: {lines[line_idx+1][64:72]}")
        print(f"64:72: {lines[line_idx+1][64:72]}")
        print(f"72:80: {lines[line_idx+1][72:80]}")

        print(f"Ángulo de fase utiliza 3 digitos y 5 decimales (9 posiciones):")
        print(f"En las columnas [31:40]: {lines[line_idx+1][31:40]}")

In [ ]:
def readPL4(pl4file: str):
    """This function extracts signals from a PL4 file and saves
    it as a CSV file. Function found at: https://github.com/ldemattos/readPL4.
    'Describe and read ATP's (Alternative Transients Program) PL4 binary data file format by means of reverse engineering.'

    Parameters
    ----------
    pl4file : str
        Name of .pl4 file
    """
    miscData = {"deltat": 0.0, "nvar": 0, "pl4size": 0, "steps": 0, "tmax": 0.0}

    with open("SCENARIOS_ATP\\" + pl4file, "rb") as f:

        pl4 = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)

        # Read DELTAT
        miscData["deltat"] = struct.unpack("<f", pl4[40:44])[0]
        DT = miscData["deltat"]

        # Read number of vars
        miscData["nvar"] = struct.unpack("<L", pl4[48:52])[0] / 2
        nv = int(miscData["nvar"])

        # Read PL4 disk size
        miscData["pl4size"] = struct.unpack("<L", pl4[56:60])[0] - 1

        # Compute the number of simulation miscData['steps'] from the PL4's file size
        miscData["steps"] = (miscData["pl4size"] - 5 * 16 - miscData["nvar"] * 16) / (
            (miscData["nvar"] + 1) * 4
        )
        step = int(miscData["steps"])

        miscData["tmax"] = (miscData["steps"] - 1) * miscData["deltat"]

        # generate pandas dataframe to store PL4's header
        dfHEAD = pd.DataFrame(columns=["TYPE", "FROM", "TO"])

        for i in range(0, nv):
            pos = 5 * 16 + i * 16
            h = struct.unpack("3x1c6s6s", pl4[pos : pos + 16])
            dfHEAD = dfHEAD.append(
                {"TYPE": int(h[0]), "FROM": h[1], "TO": h[2]}, ignore_index=True
            )
            dfHEAD["TYPE"] = dfHEAD["TYPE"].apply(lambda x: "V-node" if x == 4 else x)
            dfHEAD["TYPE"] = dfHEAD["TYPE"].apply(lambda x: "E-bran" if x == 7 else x)
            dfHEAD["TYPE"] = dfHEAD["TYPE"].apply(lambda x: "V-bran" if x == 8 else x)
            dfHEAD["TYPE"] = dfHEAD["TYPE"].apply(lambda x: "I-bran" if x == 9 else x)

        data = np.memmap(
            f,
            dtype=np.float32,
            mode="r",
            shape=(step, nv + 1),
            offset=5 * 16 + nv * 16,
        )
        currentDirectory = os.getcwd()
        file_path = currentDirectory + "\\SCENARIOS_ATP\\"
        initial = time.time()
        archivopl4 = pl4file

        # ------------ CONVERT DATA TO CSV--------------#
        header_1 = dfHEAD.iloc[:, 0].to_list()
        header_2 = dfHEAD.iloc[:, 1].to_list()
        header_3 = dfHEAD.iloc[:, 2].to_list()

        header_1.insert(0, "time".ljust(6))
        header_2.insert(0, "".ljust(6))
        header_3.insert(0, "".ljust(6))

        header = pd.MultiIndex.from_arrays([header_1, header_2, header_3])
        data = pd.DataFrame(data, columns=header)

        csv_path = currentDirectory + "\\ARCHIVOS CSV"
        path_array = csv_path.split("\\")
        s = ""

        for x in path_array:
            s = s + x
            try:
                os.stat(s)
            except Exception as e:
                os.mkdir(s)

            s = s + "\\"
        csv_name = csv_path + "\\" + archivopl4.split(".")[0] + ".csv"
        data.to_csv(csv_name, sep=";", index=False)
        final = time.time()

In [ ]:
help(readPL4)

In [ ]:
btn_dict = {"Check01": checked}


if Check01.isChecked() == True:
    for i in buses:
        for x in np.around(np.arange(Ri, Rf + D, D), 5):
            f.write("Fault01_B" + i + "_RF" + str(x) + ".atp\n")
            f.close


In [42]:
buses = [801, 815, 900]
impedances = [0, 20, 40, 60, 80, 100]


class Check:
    Check01 = False
    Check02 = False
    Check03 = True
    Check04 = True
    Check05 = False
    Check06 = False
    Check07 = True
    Check08 = False
    Check09 = False
    Check10 = False


check = Check()
fault_checkbox = [
    check.Check01,
    check.Check02,
    check.Check03,
    check.Check04,
    check.Check05,
    check.Check06,
    check.Check07,
    check.Check08,
    check.Check09,
    check.Check10,
]

bus_impedance_gen = ((bus, Z) for bus in buses for Z in impedances)

# for i, j in z:
#     print(i, j)

for idx, checkbox in enumerate(fault_checkbox):
    if checkbox:
        for bus, Z in bus_impedance_gen:
            print(f"fault{idx+1:02}_B{bus}_RF{Z}.atp")

import numpy as np


def fault_list_creator(
    impedances: np.ndarray, bus_impedance_list: list, fault_checkbox: list
):
    for idx, checkbox in enumerate(fault_checkbox):
        print(checkbox.isChecked())
        if checkbox.isChecked():
            for bus, z in bus_impedance_gen:
                f.write(f"Fault{idx+1:02}_B{bus}_RF{z}.atp\n")
                print(f"Fault{idx+1:02}_B{bus}_RF{z}.atp\n")
                f.close

fault03_B801_RF0.atp
fault03_B801_RF20.atp
fault03_B801_RF40.atp
fault03_B801_RF60.atp
fault03_B801_RF80.atp
fault03_B801_RF100.atp
fault03_B815_RF0.atp
fault03_B815_RF20.atp
fault03_B815_RF40.atp
fault03_B815_RF60.atp
fault03_B815_RF80.atp
fault03_B815_RF100.atp
fault03_B900_RF0.atp
fault03_B900_RF20.atp
fault03_B900_RF40.atp
fault03_B900_RF60.atp
fault03_B900_RF80.atp
fault03_B900_RF100.atp


In [45]:
a = ((idx + 1, checkbox) for idx, checkbox in enumerate(fault_checkbox) if checkbox)


In [46]:
for i in a:
    print(i)

(3, True)
(4, True)
(7, True)


In [44]:
with open("IEEE34_form1_loads.atp", "r+") as f:
    lines = f.readlines()
lines_copy = lines.copy()
for idx, line in enumerate(lines_copy):
    if 'C Load' in line:
        if '3f' in line:
            print(line, end='')
            print(f" Fase A carga trifásica/bifásica: {lines_copy[idx + 1]}", end='')
            print(f" Fase B carga trifásica/bifásica: {lines_copy[idx + 2]}", end='')
            print(f" Fase C carga trifásica/bifásica: {lines_copy[idx + 3]}", end='')
        else:
            print(line, end='')
            print(f" Fase A carga monofásica: {lines_copy[idx + 1]}", end='')



C Load 860 3fy
 Fase A carga trifásica/bifásica:   X0072A                      6300.9146341    5040.7317073                     0
 Fase B carga trifásica/bifásica:   X0072B                      6300.9146341    5040.7317073                     0
 Fase C carga trifásica/bifásica:   X0072C                      6300.9146341    5040.7317073                     0
C Load 840 3fy
 Fase A carga trifásica/bifásica:   N840A                       14307.923077    11128.384615                     0
 Fase B carga trifásica/bifásica:   N840B                       14307.923077    11128.384615                     0
 Fase C carga trifásica/bifásica:   N840C                       14307.923077    11128.384615                     0
C Load 844 3f
 Fase A carga trifásica/bifásica:   N844A                       953.86153846    741.89230769                     0
 Fase B carga trifásica/bifásica:   N844B                       953.86153846    741.89230769                     0
 Fase C carga trifásica/bifásica:   

In [41]:
a = 'Load 822 '
if 'A' in a:
    print(a)